In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive.metadata.readonly"]


def main():
  """Shows basic usage of the Drive v3 API.
  Prints the names and ids of the first 10 files the user has access to.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("drive", "v3", credentials=creds)

    # Call the Drive v3 API
    results = (
        service.files()
        .list(pageSize=10, fields="nextPageToken, files(id, name)")
        .execute()
    )
    items = results.get("files", [])

    if not items:
      print("No files found.")
      return
    print("Files:")
    for item in items:
      print(f"{item['name']} ({item['id']})")
  except HttpError as error:
    # TODO(developer) - Handle errors from drive API.
    print(f"An error occurred: {error}")


if __name__ == "__main__":
  main()

In [ ]:
from google.colab import drive, userdata

comet_key = userdata.get('comet_key')
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/developer-sidani/CLIPTrans.git

In [ ]:
%cd /content/CLIPTrans

In [ ]:
!git submodule update --init --recursive

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda env create --file environment.yml

In [ ]:
!conda env update -n base -f environment.yml

In [ ]:
PIP_PATH = '/usr/local/envs/cliptrans/bin/pip'
PYTHON_PATH= '/usr/local/envs/cliptrans/bin/python'

In [ ]:
%cd CLIPTrans

In [ ]:
!$PIP_PATH install -r requirements.txt

In [ ]:
!cd transformers && $PIP_PATH install -e .

In [ ]:
!cd data/multi30k && chmod +x setup.sh && ./setup.sh

### Pre-trained Models and Inference

In [ ]:
!$PYTHON_PATH src/main.py --num_gpus 1 \
    --mn multi30k --src_lang en --tgt_lang fr --prefix_length 10 \
    --bs 32 --test_ds 2016 flickr --stage translate --test \
    --lm model_best_test.pth \
    --comet_key $comet_key --comet_project CLIPTrans

### Training

#### Stage 1

In [ ]:
!$PYTHON_PATH src/main.py --num_gpus 1 --mn multi30k --prefix_length 10 --bs 32 --update_count 4 --lr 1e-5 --test_ds 2016 val --stage caption --tgt_lang fr

#### Stage 2

In [ ]:
!$PYTHON_PATH src/main.py --num_gpus 1 --mn multi30k --prefix_length 10 --bs 32 --update_count 4 --lr 1e-5 --test_ds 2016 val --stage translate --tgt_lang fr --lm model_pretrained.pth